In [1]:
import math
import os
from enum import Enum

import numpy as np
from pxr import Usd, UsdGeom

import warp as wp
import warp.examples
import warp.sim
import warp.sim.render

In [2]:
usd_stage = Usd.Stage.Open(os.path.join("./tire_models/Single_L_Front.usd"))
for prim in usd_stage.Traverse():
    print(prim.GetPath())
usd_geom = UsdGeom.Mesh(usd_stage.GetPrimAtPath("/World/Single_L_Front/Object066"))

mesh_points = np.array(usd_geom.GetPointsAttr().Get())
mesh_indices = np.array(usd_geom.GetFaceVertexIndicesAttr().Get())

mesh = wp.sim.Mesh(mesh_points, mesh_indices)
print(mesh)

/World
/World/Single_L_Front
/World/Single_L_Front/Object066
/World/Single_L_Front/Object067
/World/Single_L_Front/Object068
Warp 1.2.2 initialized:
   CUDA Toolkit 11.8, Driver 12.3
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA GeForce RTX 4090" (24 GiB, sm_89, mempool enabled)
   Kernel cache:
     /home/kbianco/.cache/warp/1.2.2
Module warp.sim.inertia 0401675 load on device 'cuda:0' took 1.21 ms


In [5]:
friction_coefficients = {
    'dry_asphalt': 1.0,
    'wet_asphalt': 0.7,
    'dry_concrete': 1.0,
    'wet_concrete': 0.65,
    'ice': 0.1,
    'snow': 0.2,
    'gravel': 0.6,
    'sand': 0.4,
    'mud': 0.4,
    'grass': 0.35,
    'rubbered_asphalt': 1.1  # Asphalt with rubber buildup from racing
}   

In [6]:
import numpy as np

# Placeholder values for empirical parameters
alpha = 1.1
beta = 1.2
gamma = 0.8
delta = 0.02
k = 1e-5  # Placeholder value for the wear coefficient

# Reference values (example)
F_z_ref = 1000.0  # Newtons
v_ref = 30.0  # m/s
T_ref = 25.0  # Celsius

# Current values (example)
F_z = 204.0  # Normal load on the tire
v = 32.0  # Vehicle speed
mu = friction_coefficients['dry_asphalt']  # Coefficient of friction
T = 30.0  # Tire temperature

# Calculate the wear rate
wear_rate = k * (F_z / F_z_ref)**alpha * (v / v_ref)**beta * mu**gamma * np.exp(-delta * (T - T_ref))

print("Tire Wear Rate:", wear_rate)

Tire Wear Rate: 1.7013677811446807e-06


In [7]:
T_air = 25.0  # Ambient air temperature

# Thermal properties of the tire
m_tire = 10.0  # kg, mass of the tire
c_tire = 900.0  # J/(kg*K), specific heat capacity of the tire
h_convection = 50.0  # W/(m^2*K), convective heat transfer coefficient
A_tire = 2.0  # m^2, surface area of the tire

# Initial tire temperature
T_tire = 30.0  # Initial temperature of the tire

# Simulation parameters
slip_ratio = 0.1  # Slip ratio of the tire
time_step = 1.0  # seconds
simulation_time = 100.0  # Total simulation time in seconds

# Function to calculate heat generation
def calculate_heat_generation(F_z, mu, v, slip_ratio):
    return mu * F_z * v * slip_ratio

# Function to calculate heat dissipation
def calculate_heat_dissipation(h_convection, A_tire, T_tire, T_air):
    return h_convection * A_tire * (T_tire - T_air)

# Simulation loop
time = 0.0
while time < simulation_time:
    # Calculate heat generation
    Q_friction = calculate_heat_generation(F_z, mu, v, slip_ratio)
    
    # Calculate temperature increase due to heat generation
    delta_T_gen = Q_friction / (m_tire * c_tire)
    
    # Update tire temperature
    T_tire += delta_T_gen
    
    # Calculate heat dissipation
    Q_dissipation = calculate_heat_dissipation(h_convection, A_tire, T_tire, T_air)
    
    # Calculate temperature decrease due to heat dissipation
    delta_T_diss = Q_dissipation * time_step / (m_tire * c_tire)
    
    # Update tire temperature
    T_tire -= delta_T_diss
    
    # Update time
    time += time_step
    
    # Print current tire temperature
    print(f"Time: {time} s, Tire Temperature: {T_tire:.2f} °C")

Time: 1.0 s, Tire Temperature: 30.02 °C
Time: 2.0 s, Tire Temperature: 30.03 °C
Time: 3.0 s, Tire Temperature: 30.05 °C
Time: 4.0 s, Tire Temperature: 30.06 °C
Time: 5.0 s, Tire Temperature: 30.08 °C
Time: 6.0 s, Tire Temperature: 30.09 °C
Time: 7.0 s, Tire Temperature: 30.11 °C
Time: 8.0 s, Tire Temperature: 30.12 °C
Time: 9.0 s, Tire Temperature: 30.14 °C
Time: 10.0 s, Tire Temperature: 30.15 °C
Time: 11.0 s, Tire Temperature: 30.17 °C
Time: 12.0 s, Tire Temperature: 30.18 °C
Time: 13.0 s, Tire Temperature: 30.20 °C
Time: 14.0 s, Tire Temperature: 30.21 °C
Time: 15.0 s, Tire Temperature: 30.22 °C
Time: 16.0 s, Tire Temperature: 30.24 °C
Time: 17.0 s, Tire Temperature: 30.25 °C
Time: 18.0 s, Tire Temperature: 30.27 °C
Time: 19.0 s, Tire Temperature: 30.28 °C
Time: 20.0 s, Tire Temperature: 30.29 °C
Time: 21.0 s, Tire Temperature: 30.30 °C
Time: 22.0 s, Tire Temperature: 30.32 °C
Time: 23.0 s, Tire Temperature: 30.33 °C
Time: 24.0 s, Tire Temperature: 30.34 °C
Time: 25.0 s, Tire Temper

In [3]:
class Tire:
    def __init__(self, mesh_points, mesh_indices, initial_grip, initial_heat):
        self.mesh_points = np.array(mesh_points, dtype=np.float32).reshape(-1, 3)
        self.mesh_indices = np.array(mesh_indices, dtype=np.int32).flatten()
        self.initial_grip = initial_grip
        self.grip = initial_grip
        self.heat = initial_heat
        self.mesh = wp.sim.Mesh(self.mesh_points, self.mesh_indices)

    def simulate_wear(self, wear_rate, heat_rate, time_step):
        # Decrease grip and increase heat
        self.grip -= wear_rate * time_step
        self.heat += heat_rate * time_step
        if self.grip < 0:
            self.grip = 0

        # Update visual and physical properties
        self.update_mesh()
        self.update_visuals()

    def update_mesh(self):
        # Update mesh points based on heat (for deformation)
        deformation_factor = 0.001 * self.heat  # Example deformation
        for i in range(len(self.mesh_points)):
            self.mesh_points[i] += deformation_factor * np.random.randn(3)
        
        # Update the mesh in Warp
        self.mesh = wp.sim.Mesh(self.mesh_points, self.mesh_indices)
        print("Mesh updated with new heat deformation:", self.heat)

    def update_visuals(self):
        # Placeholder for visual updates, e.g., changing texture or color to show wear
        print("Visuals updated for tire wear and heat buildup.")

# Example usage
# Assume mesh_points and mesh_indices are defined
initial_grip = 1.0  # Full grip
initial_heat = 0.0  # No initial heat

tire = Tire(mesh_points, mesh_indices, initial_grip, initial_heat)

# Simulate wear over time
time_step = 0.1  # Each step is 0.1 seconds
wear_rate = 0.05  # Grip wears at this rate
heat_rate = 0.02  # Heat increases at this rate

for t in np.arange(0, 5.0, time_step):  # Simulate for 5 seconds
    tire.simulate_wear(wear_rate, heat_rate, time_step)

Mesh updated with new heat deformation: 0.002
Visuals updated for tire wear and heat buildup.
Mesh updated with new heat deformation: 0.004
Visuals updated for tire wear and heat buildup.
Mesh updated with new heat deformation: 0.006
Visuals updated for tire wear and heat buildup.
Mesh updated with new heat deformation: 0.008
Visuals updated for tire wear and heat buildup.
Mesh updated with new heat deformation: 0.01
Visuals updated for tire wear and heat buildup.
Mesh updated with new heat deformation: 0.012
Visuals updated for tire wear and heat buildup.
Mesh updated with new heat deformation: 0.014
Visuals updated for tire wear and heat buildup.
Mesh updated with new heat deformation: 0.016
Visuals updated for tire wear and heat buildup.
Mesh updated with new heat deformation: 0.018000000000000002
Visuals updated for tire wear and heat buildup.
Mesh updated with new heat deformation: 0.020000000000000004
Visuals updated for tire wear and heat buildup.
Mesh updated with new heat defo